# Introdução à filtragem colaborativa baseada em vizinhança
## Aula 02: Cálculo da Similaridade

In [1]:

import pandas as pd
import numpy as np
import plotly.express as px
from rich.console import Console
from rich.table import Table

### Matriz de avaliações

In [2]:
class FancyMatrix:
    ''' Matriz 
    '''
    def __init__(self, num_lines: int, num_columns: int):
        self.line_index = {}
        self.column_index = {}
        self.data = np.zeros((num_lines, num_columns), dtype=np.float16)
    
    def __setitem__(self, key: tuple, value: np.float16):
        i,j = key
        if isinstance(key, tuple):
            line, column = key
            if line not in self.line_index:
                self.line_index[line] = len(self.line_index.keys())
            if column not in self.column_index:
                self.column_index[column] = len(self.column_index.keys())

            i = self.line_index[line]
            j = self.column_index[column]
            self.data[i,j] = value
        else:
            raise KeyError("Key must be a tuple")
    
    def print(self, title="Table"):
        table = Table(title=title)
        table.add_column("", justify="center", style="cyan", no_wrap=True)
        for col in self.column_index.keys():
            table.add_column(col, justify="center")
        for line in self.line_index.keys():
            line_index = self.line_index[line]
            #line name + the float values
            row = [line]+["-" if i==0 else format(i, ".2f")   for i in self.data[line_index]]
            #row_str = list(map(lambda x: '-' if x==0 else str(x), row))
            table.add_row(*row)
        console = Console()
        console.print(table)
    
    def __getitem__(self, key):
        ''' Retorna um elemento ou uma linha representando.
        
        Args:
          key : int ou tuple de inteiros (i,j)
        '''
        if isinstance(key, tuple):
            line, column = key
            if line not in self.line_index:
                raise KeyError(f"The line key {line} has not been found in the index.")
            if column not in self.column_index:
                raise KeyError(f"The column key {column} has not been found in the index.")
            i = self.line_index[line]
            j = self.column_index[column]
            return self.data[i,j]
        line = key
        if line not in self.line_index:
            raise KeyError(f"The line key {line} has not been found in the index.")
        i = self.line_index[line]
        return self.data[i]
    def toStr(self):
        return ','.join(self.line_index.keys())+'\n'+str(self.data) 
    
    def __str__(self) -> str:
        return self.toStr()
      
    def __repr__(self) -> str:
        table = Table(title="")
        table.add_column("", justify="center", style="cyan", no_wrap=True)
        for col in self.column_index.keys():
            table.add_column(col, justify="center")
        for line in self.line_index.keys():
            line_index = self.line_index[line]
            #line name + the float values
            row = [line]+["-" if i==0 else format(i, ".2f")   for i in self.data[line_index]]
            #row_str = list(map(lambda x: '-' if x==0 else str(x), row))
            table.add_row(*row)
        console = Console()
        with console.capture() as capture:
            console.print(table)

        return capture.get()
        #return self.toStr()      
          
    
class SingleRatingMatrix(FancyMatrix):
    '''Representa uma matrix de avaliação. 
    
    As linhas indicam os usuários e nas colunas os itens. 
    Os usuários e os itens podem ser quaisquer elementos imutáveis (hasheable). 
    Mantemos um índice para cada um deles, que transforma o item ou usuário em um índice da matriz.
    A matriz é um numpy array. 
    '''
    def __init__(self, num_of_users, num_of_items):
        super().__init__(num_of_users, num_of_items)
        
    def get_user_index(self):
        return self.line_index
    
    def get_item_index(self):
        return self.column_index
    
    def get_all_user_ratings_for(self, item:int) -> np.array:
        ''' Retorna o vetor de avaliações dos usuários para um determinado item. 
        '''
        item_index = self.get_item_index()
        if item not in item_index:
            raise KeyError(f"Item {item} has not been found in the index.")
        i = item_index[item]
        return self.data[:,i]
            
    def get_index_of_user(self, user):
        if user in self.line_index:
            return self.line_index[user]
        raise KeyError(f"Não há o usuário {user} no índice de usuários. Ele fez alguma avaliação?")
      
    def normalize(self):
        '''Normaliza a matriz de avaliações subtraindo de cada avaliação a média da avaliação do usuário.
           Para o cálculo da média, não se considera os valores iguais a zero e apenas com usuários que avaliaram pelo menos 2 itens.
        '''
        for user_index in self.get_user_index().values():
            sum_items = self.data[user_index,:].sum()
            non_zero_count = np.count_nonzero(self.data[user_index,:])
            #print(f"user={user_index}, sum_items={sum_items}, non_zero_count={non_zero_count}")
            # Normalizar um vetor com apenas um elemento não nulo vai zerá-lo.
            if non_zero_count<2:
                continue
            non_zero_items = self.data[user_index,:].nonzero()
            self.data[user_index,non_zero_items] -=  sum_items/non_zero_count
        return self

    
    @staticmethod
    def build_from_dataframe(ratings_df:pd.DataFrame, **kargs):
        '''Builds a ratings matrix from a pandas dataframe.
        
        Args
        ratings_df : pandas.DataFrame
            Dataframe containing the ratings.
        
        Allowed Keyword arguments are:
          item_column : str, optional
            Name of the column containing the items. The default is "WineID".
          user_column : str, optional
            Name of the column containing the users. The default is "UserID".

        Returns
          SingleRatingMatrix: Matrix containing the ratings.
        '''
        item_column=kargs['item_column'] if 'item_column' in kargs else "WineID"
        user_column=kargs['user_column'] if 'user_column' in kargs else "UserID"
        rating_column=kargs['rating_column'] if 'rating_column' in kargs else "Rating"

        
        item_ids = ratings_df[item_column].unique()
        user_ids = ratings_df[user_column].unique()
        qty_items = len(item_ids)
        qty_users = len(user_ids)
        M = SingleRatingMatrix(qty_users, qty_items)
        #display(user_ids)
        for i in range(qty_users):
            user = user_ids[i]
            #print("Fetching the ratings of user %d" % user)
            ratings_user = ratings_df[ratings_df[user_column]==user]
            #print(ratings_user)
            for index, row in ratings_user.iterrows():
                item = row[item_column]
                rating = row[rating_column]
                #print(wine, rating)
                M[user, item] = rating
        return M


In [6]:
# Testando a matriz de avaliações em um df simples
df_test = pd.read_csv('simple.csv')
#df_test
M = SingleRatingMatrix.build_from_dataframe(df_test, user_column ='UserID', item_column='ItemID')
M.print("Matriz de avaliações")
M.normalize()
#.print("Matriz de avaliações (normalizada)")
M.print("Matriz de avaliações normalizada")


                            Matriz de avaliações                            
┏━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┓
┃           ┃ Cella ┃ Black Tower ┃ Alorna ┃ Reservado ┃ Gato Negro ┃ Toro ┃
┡━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━┩
│ Severino  │ 5.00  │    3.00     │   -    │   2.00    │    2.00    │ 2.00 │
│   James   │ 4.00  │    3.00     │  4.00  │     -     │    3.00    │ 3.00 │
│ Tarantino │ 5.00  │    2.00     │  5.00  │   2.00    │    1.00    │ 1.00 │
│   Helio   │ 3.00  │    5.00     │  3.00  │     -     │    1.00    │ 1.00 │
│   Pedro   │ 3.00  │    3.00     │  3.00  │   2.00    │    4.00    │ 5.00 │
│   Elias   │ 2.00  │    3.00     │  2.00  │   3.00    │    5.00    │ 5.00 │
└───────────┴───────┴─────────────┴────────┴───────────┴────────────┴──────┘

                      Matriz de avaliações normalizada                       
┏━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃           ┃ Cella ┃ Black Tower ┃ Alorna ┃ Reservado ┃ Gato Negro ┃ Toro  ┃
┡━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ Severino  │ 2.20  │    0.20     │   -    │   -0.80   │   -0.80    │ -0.80 │
│   James   │ 0.60  │    -0.40    │  0.60  │     -     │   -0.40    │ -0.40 │
│ Tarantino │ 2.33  │    -0.67    │  2.33  │   -0.67   │   -1.67    │ -1.67 │
│   Helio   │ 0.40  │    2.40     │  0.40  │     -     │   -1.60    │ -1.60 │
│   Pedro   │ -0.33 │    -0.33    │ -0.33  │   -1.33   │    0.67    │ 1.67  │
│   Elias   │ -1.33 │    -0.33    │ -1.33  │   -0.33   │    1.67    │ 1.67  │
└───────────┴───────┴─────────────┴────────┴───────────┴────────────┴───────┘

### Vamos calcular a matriz de similaridades entre os itens
#### Vamos usar a função de similaridade de cosseno ajustado
Lembre-se que $nr_{i,u} = r_{i,u} - \bar{r}_u$
$$ Sim(a, b) = \frac{\sum_{u} nr_{a,u}nr_{b, u}}{\sqrt{\sum_{u}nr_{a,u}^2} \sqrt{\sum_{u}nr_{b,u}^2}} $$

In [8]:

class ItemSimMatrix(FancyMatrix):
    def __init__(self, number_of_items):
        self.verbose = False
        super().__init__(number_of_items, number_of_items)
    
    def get_item_index(self):
        return self.line_index
    
    def item_to_index(self, item):
        return self.get_item_index()[item]
    
    def index_to_item(self, item_index):
        return list(self.get_item_index().keys())[item_index]
       
    def __setitem__(self, key, value: np.float16):
        if isinstance(key, tuple):
            item_a, item_b = key
            super().__setitem__((item_a, item_b), value)
            super().__setitem__((item_b, item_a), value)            
        else:
            raise KeyError("Key must be a tuple")
        
    @staticmethod
    def calc_adjusted_cos_sim_nozero(normalized_ratings_item_a, normalized_ratings_item_b):
        '''Cálculo da similaridade porém levando em consideração apenas os pares onde houve avaliação. 
        '''
        #Primeiro os itens onde é zero (nao há avaliação) não entram na conta.
        # Assim, trabalhamos apenas com os valores não zerados.
        
        non_zero_indexes_of_item_a = set(np.flatnonzero(normalized_ratings_item_a))
        non_zero_indexes_of_item_b = set(np.flatnonzero(normalized_ratings_item_b))
        non_zero_indexes = list(non_zero_indexes_of_item_a & non_zero_indexes_of_item_b)
        #print(f" Selected indexes: {non_zero_indexes}")
        normalized_ratings_item_a = normalized_ratings_item_a[non_zero_indexes]
        normalized_ratings_item_b = normalized_ratings_item_b[non_zero_indexes]
        
        
        den_part_a = np.sqrt(sum(np.square(normalized_ratings_item_a)))
        #print(f"Vector of item a: {normalized_ratings_item_a}")
        #print(f"Vector of item b: {normalized_ratings_item_b}")
        num = sum(np.multiply(normalized_ratings_item_a,normalized_ratings_item_b))
        den_part_b = np.sqrt(sum(np.square(normalized_ratings_item_b)))
        #print(f"Formula: {num}/({den_part_a}.{den_part_b})")
        sim_a_b = num/(den_part_a*den_part_b)
        return sim_a_b

    @staticmethod
    def build_from_single_ratings_matrix(ratings_matrix: SingleRatingMatrix, verbose=False):
        item_index = ratings_matrix.get_item_index() 
        M = ItemSimMatrix(len(item_index))
        for item_a in item_index:
            user_ratings_for_item_a = ratings_matrix.get_all_user_ratings_for(item_a)
            #den_part_a = np.sqrt(sum(np.square(user_ratings_for_item_a)))
            for item_b in item_index:
                if verbose:
                    print("--------------------------------------")
                    print(f"Calculando a similaridade entre {item_a} e {item_b}")
                user_ratings_for_item_b = ratings_matrix.get_all_user_ratings_for(item_b)
                #num = sum(np.multiply(user_ratings_for_item_a,user_ratings_for_item_b))
                #den_part_b = np.sqrt(sum(np.square(user_ratings_for_item_b)))
                sim_a_b = ItemSimMatrix.calc_adjusted_cos_sim_nozero(user_ratings_for_item_a, user_ratings_for_item_b)
                M[item_a, item_b] = sim_a_b
                if verbose:
                    print(f"Sim entre {item_a, item_b}={sim_a_b}")
        return M
    
    

In [10]:
df_test = pd.read_csv('simple.csv')
#df_test
M = SingleRatingMatrix.build_from_dataframe(df_test, user_column ='UserID', item_column='ItemID')
SimMatrix = ItemSimMatrix.build_from_single_ratings_matrix(M.normalize(), False)

SimMatrix.print("Matriz de similaridades entre os vinhos.")
#print(M.get_item_index())
#['Cella', 'Black Tower', 'Alorna', 'Reservado', 'Gato Negro', 'Toro']
#SimMatrix.index_to_item(5)
#SimMatrix['Black Tower']




                   Matriz de similaridades entre os vinhos.                    
┏━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃             ┃ Cella ┃ Black Tower ┃ Alorna ┃ Reservado ┃ Gato Negro ┃ Toro  ┃
┡━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│    Cella    │ 1.00  │    0.02     │  1.00  │   -0.40   │   -0.82    │ -0.76 │
│ Black Tower │ 0.02  │    1.00     │ -0.04  │   0.58    │   -0.45    │ -0.44 │
│   Alorna    │ 1.00  │    -0.04    │  1.00  │   -0.16   │   -0.87    │ -0.81 │
│  Reservado  │ -0.40 │    0.58     │ -0.16  │   1.00    │    0.07    │ -0.20 │
│ Gato Negro  │ -0.82 │    -0.45    │ -0.87  │   0.07    │    1.00    │ 0.96  │
│    Toro     │ -0.76 │    -0.44    │ -0.81  │   -0.20   │    0.96    │ 1.00  │
└─────────────┴───────┴─────────────┴────────┴───────────┴────────────┴───────┘

'Toro'

### Problema poucas avaliações correspondentes 

Lembre-se que $nr_{i,u} = r_{i,u} - \bar{r}_u$
$$ Sim(a, b) = \frac{\sum_{u} nr_{a,u}nr_{b, u}}{\sqrt{\sum_{u}nr_{a,u}^2} \sqrt{\sum_{u}nr_{b,u}^2}} $$

Imagine que apenas um usuário avaliou dois itens (a e b) e de maneira bem distinta: 1 e 5 então teríamos:
$$ Sim(a,b) = \frac{1 \times 5}{\sqrt{1^2}\sqrt{5^2}} = 1.0$$

A fórmula vai dizer que são pares perfeitos! 
#### Necessidade de estabelecer um limiar 
Vamos inserir um limiar que vai indicar o número mínimo de avaliações de índice correspondente no vetor (overlap) como um parâmetro do nosso sistema.


### Plot do número de avaliações da base X-Wines

In [6]:
# Vamos abrir o dataframe
#wine_ratings = pd.read_csv('../data/XWines_Test_1K_ratings.csv', low_memory=False)
wine_ratings = pd.read_csv('../data/XWines_Slim_150K_ratings.csv', low_memory=False)
wine_ratings.head()

#Vamos dar uma olhadinha na contagem das avaliações
df = wine_ratings
# Contando os Usuários por seus IDs já que um usuário pode dar mais de uma avaliação.
data = df['UserID'].value_counts()
data
# Crie o histograma
fig = px.histogram(data, nbins=67)
fig.update_layout(
    xaxis_title="Número de Itens Avaliados",
    yaxis_title="Número de Usuários",
)
# Ajustando para categoria para não exibir números quebrados no eixo x (e vazios)
#fig.update_xaxes(type='category')
fig.show()

## Aula 03 - Predição
### Selecionar vizinhança

Top-K, seleciona os top-k mais similares
Threshold seleciona os itens cuja similaridade é maior ou igual a um determinado threshold


In [6]:
def select_top_k(SimMatrix:ItemSimMatrix, item, k=2):
    line = list(SimMatrix[item])
    indexes = sorted(range(len(line)), key=lambda k: line[k], reverse=True)    
    indexes.remove(SimMatrix.item_to_index(item))
    sim_items = list()
    for i in indexes:
        neighbor = SimMatrix.index_to_item(i)
        sim_value = SimMatrix[item, neighbor]
        sim_items.append((neighbor, sim_value))
        k-=1
        if(k==0): 
            break
    return sim_items
#Vamos montar uma lista da vizinhança de cada item
items=['Cella', 'Black Tower', 'Alorna', 'Reservado', 'Gato Negro', 'Toro']
for item in items:
    print(f"{item}: {select_top_k(SimMatrix, item, 2)}")


Cella: [('Alorna', 1.0), ('Black Tower', 0.01761)]
Black Tower: [('Reservado', 0.58), ('Cella', 0.01761)]
Alorna: [('Cella', 1.0), ('Black Tower', -0.03845)]
Reservado: [('Black Tower', 0.58), ('Gato Negro', 0.06866)]
Gato Negro: [('Toro', 0.9585), ('Reservado', 0.06866)]
Toro: [('Gato Negro', 0.9585), ('Reservado', -0.1992)]


### Computar as predições

$$Pred(u, i) = \bar{r}_u + \frac{\sum_{j \in N_i}(sim(i,j) \times r_{u,j})}{\sum_{j \in N_i}sim(i,j)}$$

Onde: 
- $\bar{r}_u$ é avaliação média do usuário u (sem normalização)
- $r_{u,j}$ é a avaliação do item j feita pelo usuário ativo u
- $N_i$ é a vizinhança contendo os na vizinhança que usuário o usuário $u$ avaliou
- $sim(i,j)$ é a similaridade entre o item i e j
- $Pred(u, i)$ é a predição da avaliação do item i para o usuário u



In [10]:
import copy
class RatingPredictor:
    def __init__(self, sim_matrix:ItemSimMatrix, rating_matrix: SingleRatingMatrix):
        self.sim_matrix = sim_matrix
        self.rating_matrix = rating_matrix
        self.norm_rating_matrix = copy.deepcopy(rating_matrix)
        self.norm_rating_matrix.normalize()
        
    def get_neighborhood(self, item, k=2):
        SimMatrix = self.sim_matrix
        line = list(SimMatrix[item])
        indexes = sorted(range(len(line)), key=lambda k: line[k], reverse=True)    
        indexes.remove(SimMatrix.item_to_index(item))
        sim_items = list()
        sim_values = list()
        sim_index = list()
        for i in indexes:
            neighbor = SimMatrix.index_to_item(i)
            sim_value = SimMatrix[item, neighbor]
            sim_values.append(sim_value)
            sim_items.append((neighbor, sim_value, i))
            sim_index.append(i)
            k-=1
            if(k==0): 
                break
        return sim_items, sim_index, sim_values

    def get_indexes_of_items_rated_by(self, user)->list:
        rating_line = self.rating_matrix[user]
        #print(rating_line)
        non_zero_indexes = list(np.flatnonzero(rating_line))
        return non_zero_indexes
        
    def get_user_avg(self, user):
        rating_line = self.rating_matrix[user]
        #print(rating_line)
        non_zero_index = self.get_indexes_of_items_rated_by(user)
        #print(non_zero_indexes)
        #print(f" Selected indexes: {non_zero_indexes}")
        
        ratings_count = len(non_zero_index)
        avg = 0.0
        if len(non_zero_index)>0:
            avg = sum(rating_line[non_zero_index])/ratings_count
        return avg

    def predict(self, u, i, **kargs):
        verbose=kargs.get('verbose',False)
        avg_user = self.get_user_avg(u)
        if verbose:
            print(f"Avg rating of {u}:{avg_user}")
        
        N, index, sim_values = self.get_neighborhood(i, 2)
        if verbose:
            print(f"Vizinhança do item {i}:{N}")
            print(f"Items rated by user {u}: {self.get_indexes_of_items_rated_by(u)}")
        neighbor_items_rated_by_user = list(set(self.get_indexes_of_items_rated_by(u)) & set(index))
        if verbose:
            print(f"Neighbor items also rated by user: {neighbor_items_rated_by_user}")
        num_debug = [f"sim({i},{j}):{self.sim_matrix[i, j]} X r_({u},{j}):{self.norm_rating_matrix[u, j]}" for (j, sim, index) in N if index in neighbor_items_rated_by_user ]
        num = sum([self.sim_matrix[i, j]*self.norm_rating_matrix[u, j] for (j, _, index) in N if index in neighbor_items_rated_by_user])
        
        if verbose:
            print(f"Numerator ({num}) = {num_debug}")

        den = sum([self.sim_matrix[i, j] for (j, _, index) in N if index in neighbor_items_rated_by_user])
        den_debug = "+".join([f"sim({i},{j}):{self.sim_matrix[i, j]}" for (j, _, index) in N if index in neighbor_items_rated_by_user])

        if verbose:
            print(f"Denominator ({den}) = {den_debug}")
        
        return avg_user+ num/den
        

df_test = pd.read_csv('simple.csv')
#df_test
RatingsMatrix = SingleRatingMatrix.build_from_dataframe(df_test, user_column ='UserID', item_column='ItemID')
SimMatrix = ItemSimMatrix.build_from_single_ratings_matrix(M.normalize())

pred = RatingPredictor(SimMatrix, RatingsMatrix)
#Pelos calculos manuais ficou 3. alguma coisa
#pred.predict('Helle', 'Star Trek')

pred.predict('Helio', 'Reservado', verbose=False)
pred.predict('Helio', 'Reservado', verbose=False)


4.5759058823529415

In [12]:
pred.predict('Helio', 'Reservado', verbose=True)

Avg rating of Helio:2.6
Vizinhança do item Reservado:[('Black Tower', 0.5796, 1), ('Gato Negro', 0.0689, 4)]
Items rated by user Helio: [0, 1, 2, 4, 5]
Neighbor items also rated by user: [1, 4]
Numerator (1.2813720703125) = ['sim(Reservado,Black Tower):0.57958984375 X r_(Helio,Black Tower):2.400390625', 'sim(Reservado,Gato Negro):0.06890869140625 X r_(Helio,Gato Negro):-1.599609375']
Denominator (0.64849853515625) = sim(Reservado,Black Tower):0.57958984375+sim(Reservado,Gato Negro):0.06890869140625


4.5759058823529415